# Steps to reproduce experiments in :
## **Reproducible evaluation of classification methods in Alzheimer's disease: Framework and application to MRI and PET data**

## Prerequisites

The original [ADNI](http://adni.loni.usc.edu/), [AIBL](https://aibl.csiro.au/research/neuroimaging/) and [OASIS](http://www.oasis-brains.org/) datasets should be downloaded without further touch (Data we used in our paper was downloaded in October 2016).
Fix the paths where your data is stored on your computer:

In [1]:
!export ADNI_PATH="~/Aramis/Data/ADNI"
!export AIBL_PATH="~/Aramis/Data/AIBL"
!export OASIS_PATH="~/Aramis/Data/OASIS"

In [2]:
!export OUT_PATH="~/Aramis/Data/OUTPUT"

In [3]:
!export WORKING_DIR="~/Aramis/Data/tmp/WORKING_DIR"

## 1. Convert datasets into BIDS format

In [ ]:
!clinica convert adni-to-bids $ADNI_PATH/IMAGES $ADNI_PATH/CLINICAL_DATA $OUT_PATH/ADNI/BIDS -m T1 PET_FDG
!clinica convert aibl-to-bids $AIBL_PATH/IMAGES $AIBL_PATH/CLINICAL_DATA $OUT_PATH/AIBL/BIDS
!clinica convert oasis-to-bids $OASIS_PATH/IMAGES $OASIS_PATH/CLINICAL_DATA $OUT_PATH/OASIS/BIDS

Define folders for the next steps:

In [ ]:
import os

adnimerge = 'PATH/TO/ADNIMERGE.csv'

adni_bids = os.path.join(os.environ.get('OUT_PATH'), 'ADNI/BIDS')
aibl_bids = os.path.join(os.environ.get('OUT_PATH'), 'AIBL/BIDS')
oasis_bids = os.path.join(os.environ.get('OUT_PATH'), 'OASIS/BIDS')

adni_tsv_dir = os.path.join(os.environ.get('OUT_PATH'), 'ADNI/TSV')
aibl_tsv_dir = os.path.join(os.environ.get('OUT_PATH'), 'AIBL/TSV')
oasis_tsv_dir = os.path.join(os.environ.get('OUT_PATH'), 'OASIS/TSV')

adni_caps_dir = os.path.join(os.environ.get('OUT_PATH'), 'ADNI/CAPS')
aibl_caps_dir = os.path.join(os.environ.get('OUT_PATH'), 'AIBL/CAPS')
oasis_caps_dir = os.path.join(os.environ.get('OUT_PATH'), 'OASIS/CAPS')

working_dir = os.environ.get('WORKING_DIR')


## 2. Create the subjects lists
Choose the subjects at baseline with available T1 MRI for ADNI, AIBL and OASIS, and with FDG-PET for ADNI:

In [ ]:
from Code.subjects_lists.subject_lists import run_subjects_lists

### ADNI dataset
database = 'ADNI'

# For T1
subjects_list = 'T1'
run_subjects_lists(adni_bids, adni_tsv_dir, database, subjects_list, adnimerge)

# For FDG-PET
subjects_list = 'PET'
run_subjects_lists(adni_bids, adni_tsv_dir, database, subjects_list, adnimerge)


### AIBL dataset
database = 'AIBL'
run_subjects_lists(aibl_bids, aibl_tsv_dir, database)


### OASIS dataset
database = 'OASIS'
run_subjects_lists(oasis_bids, oasis_tsv_dir, database)


## 3. Create demographic tables information
Get demographic information of the different populations:

In [ ]:
from Code.subjects_lists.lists_stats import run_lists_stats

### ADNI dataset
database = 'ADNI'

# For T1
subjects_list = 'T1'
run_lists_stats(adni_bids, adni_tsv_dir, database, subjects_list, adnimerge)

# For FDG-PET
subjects_list = 'PET'
run_lists_stats(adni_bids, adni_tsv_dir, database, subjects_list, adnimerge)


### AIBL dataset
database = 'AIBL'
run_lists_stats(aibl_bids, aibl_tsv_dir, database)


### OASIS dataset
database = 'OASIS'
run_lists_stats(oasis_bids, oasis_tsv_dir, database)


## 4. Run image processing pipelines
Used pipelines are integrated into Clinica software

In [ ]:
### ADNI T1
!clinica run t1-spm-full-prep $OUT_PATH/ADNI/BIDS $OUT_PATH/ADNI/CAPS/ ADNIbl -tsv /SUBJECTS_DIR/subjects_T1_PET.tsv -wd $WORKING_DIR -np 8
### ADNI FDG-PET
!clinica run pet-preprocess-volume $OUT_PATH/ADNI/BIDS $OUT_PATH/ADNI/CAPS/ ADNIbl -tsv $OUT_PATH/ADNI/TSV/subjects_T1_PET.tsv -wd $WORKING_DIR -np 8

### AIBL T1
!clinica run t1-spm-full-prep $OUT_PATH/AIBL/BIDS $OUT_PATH/AIBL/CAPS/ AIBLbl -tsv $OUT_PATH/AIBL/TSV/subjects_T1.tsv -wd $WORKING_DIR -np 8
### OASIS T1
!clinica run t1-spm-full-prep $OUT_PATH/OASIS/BIDS $OUT_PATH/OASIS/CAPS/ OASISbl -tsv $OUT_PATH/OASIS/TSV/subjects_T1.tsv -wd $WORKING_DIR -np 8


In [ ]:
### AIBL and OASIS are registered into ADNI template to later test ADNI trained classifiers on AIBL and OASIS

###AIBL
!clinica run t1-spm-dartel-existing-template $OUT_PATH/AIBL/BIDS $OUT_PATH/AIBL/CAPS/ ADNIbl -tsv $OUT_PATH/AIBL/TSV/subjects_T1.tsv -wd $WORKING_DIR -np 8
!clinica run t1-spm-dartel2mni $OUT_PATH/AIBL/BIDS $OUT_PATH/AIBL/CAPS/ AIBLbl -tsv $OUT_PATH/AIBL/TSV/subjects_T1.tsv -wd $WORKING_DIR -np 8

###OASIS
!clinica run t1-spm-dartel-existing-template $OUT_PATH/OASIS/BIDS $OUT_PATH/OASIS/CAPS/ ADNIbl -tsv $OUT_PATH/OASIS/SUBJECTS_DIR/subjects_T1.tsv -wd $WORKING_DIR -np 8
!clinica run t1-spm-dartel2mni $OUT_PATH/OASIS/BIDS $OUT_PATH/OASIS/CAPS/ OASISbl -tsv $OUT_PATH/OASIS/SUBJECTS_DIR/subjects_T1.tsv -wd $WORKING_DIR -np 8


## 5. Run the classification tasks

We present in more detail the experiments where main classifications are performed

### Classification results using T1-weighted MRI and FDG-PET images from ADNI dataset

(from `1-main_classifications/ADNI_run_classifications.py`)

In [ ]:
import os
from os import path
import pandas as pd
import clinica.pipeline.machine_learning.ml_workflows as ml_wf

n_iterations = 250
n_threads = 8

caps_dir = '/ADNI/CAPS'
output_dir = '/ADNI/CLASSIFICATION/OUTPUTS'

group_id = 'ADNIbl'
image_types = ['T1', 'fdg']
tasks_dir = '/ADNI/lists_by_task'
tasks = [('CN', 'AD'),
         ('CN', 'MCI'),
         ('CN', 'pMCI'),
         ('sMCI', 'pMCI'),
         ('CN-', 'AD+'),
         ('CN-', 'MCI+'),
         ('CN-', 'pMCI+'),
         ('MCI-', 'MCI+'),
         ('sMCI+', 'pMCI+')]

smoothing = [0, 4, 8, 12]
atlases = ['AAL2', 'LPBA40', 'Neuromorphometrics', 'AICHA', 'Hammers']
pvcs = [None, 'rbv']
rb_classifiers = {'linear_svm': ml_wf.RB_RepHoldOut_DualSVM,
                  'logistic_regression': ml_wf.RB_RepHoldOut_LogisticRegression,
                  'random_forest': ml_wf.RB_RepHoldOut_RandomForest}

###################################
### Voxel based classifications ###
###################################

for image_type in image_types:
    for fwhm in smoothing:
        for task in tasks:
            subjects_visits_tsv = path.join(tasks_dir, '%s_vs_%s_subjects_sessions.tsv' % (task[0], task[1]))
            diagnoses_tsv = path.join(tasks_dir, '%s_vs_%s_diagnoses.tsv' % (task[0], task[1]))
            for pvc in pvcs:
                if image_type == 'T1':
                    if pvc is None:
                        classification_dir = path.join(output_dir, image_type, 'voxel_based',
                                                       'smooothing-%s' % fwhm, 'linear_svm',
                                                       '%s_vs_%s' % (task[0], task[1]))
                    else:
                        continue
                else:
                    classification_dir = path.join(output_dir, image_type, 'voxel_based', 'pvc-%s' % pvc,
                                                   'smooothing-%s' % fwhm, 'linear_svm',
                                                   '%s_vs_%s' % (task[0], task[1]))
                if not path.exists(classification_dir):
                    os.makedirs(classification_dir)

                print "Running %s" % classification_dir
                wf = ml_wf.VB_RepHoldOut_DualSVM(caps_dir, subjects_visits_tsv, diagnoses_tsv, group_id, image_type,
                                                 classification_dir, fwhm=fwhm, pvc=pvc, n_iterations=n_iterations,
                                                 n_threads=n_threads)
                wf.run()

####################################
### Region based classifications ###
####################################

for image_type in image_types:
    for atlas in atlases:
        for task in tasks:
            subjects_visits_tsv = path.join(tasks_dir, '%s_vs_%s_subjects_sessions.tsv' % (task[0], task[1]))
            diagnoses_tsv = path.join(tasks_dir, '%s_vs_%s_diagnoses.tsv' % (task[0], task[1]))
            for classifier in rb_classifiers:
                ml_class = rb_classifiers[classifier]
                for pvc in pvcs:
                    if image_type == 'T1':
                        if pvc is None:
                            classification_dir = path.join(output_dir, image_type, 'region_based',
                                                           'atlas-%s' % atlas, classifier,
                                                           '%s_vs_%s' % (task[0], task[1]))
                        else:
                            continue
                    else:
                        classification_dir = path.join(output_dir, image_type, 'region_based', 'pvc-%s' % pvc,
                                                       'atlas-%s' % atlas, classifier,
                                                       '%s_vs_%s' % (task[0], task[1]))
                    if not path.exists(classification_dir):
                        os.makedirs(classification_dir)

                    print "Running %s" % classification_dir
                    wf = ml_class(caps_dir, subjects_visits_tsv, diagnoses_tsv, group_id, image_type, atlas,
                                  classification_dir, pvc=pvc, n_iterations=n_iterations, n_threads=n_threads)
                    wf.run()


### Classification results using T1-weighted MRI from AIBL dataset

(from `1-main_classifications/AIBL_run_classifications.py`)

In [ ]:
import os
from os import path
import pandas as pd
import clinica.pipeline.machine_learning.ml_workflows as ml_wf

n_iterations = 250
n_threads = 8

caps_dir = '/AIBL/CAPS'
output_dir = '/AIBL/CLASSIFICATION/OUTPUTS'

group_id = 'AIBLbl'
image_types = ['T1']
tasks_dir = '/AIBL/lists_by_task'
tasks = [('CN', 'AD'),
         ('CN', 'MCI'),
         ('sMCI', 'pMCI')]

smoothing = [0, 4, 8, 12]
atlases = ['AAL2', 'LPBA40', 'Neuromorphometrics', 'AICHA', 'Hammers']
pvcs = [None, 'rbv']
rb_classifiers = {'linear_svm': ml_wf.RB_RepHoldOut_DualSVM,
                  'logistic_regression': ml_wf.RB_RepHoldOut_LogisticRegression,
                  'random_forest': ml_wf.RB_RepHoldOut_RandomForest}

###################################
### Voxel based classifications ###
###################################

for image_type in image_types:
    for fwhm in smoothing:
        for task in tasks:
            subjects_visits_tsv = path.join(tasks_dir, '%s_vs_%s_subjects_sessions.tsv' % (task[0], task[1]))
            diagnoses_tsv = path.join(tasks_dir, '%s_vs_%s_diagnoses.tsv' % (task[0], task[1]))
            for pvc in pvcs:
                if image_type == 'T1':
                    if pvc is None:
                        classification_dir = path.join(output_dir, image_type, 'voxel_based',
                                                       'smooothing-%s' % fwhm, 'linear_svm',
                                                       '%s_vs_%s' % (task[0], task[1]))
                    else:
                        continue
                else:
                    classification_dir = path.join(output_dir, image_type, 'voxel_based', 'pvc-%s' % pvc,
                                                   'smooothing-%s' % fwhm, 'linear_svm',
                                                   '%s_vs_%s' % (task[0], task[1]))
                if not path.exists(classification_dir):
                    os.makedirs(classification_dir)

                print "Running %s" % classification_dir
                wf = ml_wf.VB_RepHoldOut_DualSVM(caps_dir, subjects_visits_tsv, diagnoses_tsv, group_id, image_type,
                                                 classification_dir, fwhm=fwhm, pvc=pvc, n_iterations=n_iterations,
                                                 n_threads=n_threads)
                wf.run()

####################################
### Region based classifications ###
####################################

for image_type in image_types:
    for atlas in atlases:
        for task in tasks:
            subjects_visits_tsv = path.join(tasks_dir, '%s_vs_%s_subjects_sessions.tsv' % (task[0], task[1]))
            diagnoses_tsv = path.join(tasks_dir, '%s_vs_%s_diagnoses.tsv' % (task[0], task[1]))

            for classifier in rb_classifiers:
                ml_class = rb_classifiers[classifier]

                for pvc in pvcs:
                    if image_type == 'T1':
                        if pvc is None:
                            classification_dir = path.join(output_dir, image_type, 'region_based',
                                                           'atlas-%s' % atlas, classifier,
                                                           '%s_vs_%s' % (task[0], task[1]))
                        else:
                            continue
                    else:
                        classification_dir = path.join(output_dir, image_type, 'region_based', 'pvc-%s' % pvc,
                                                       'atlas-%s' % atlas, classifier,
                                                       '%s_vs_%s' % (task[0], task[1]))

                    if not path.exists(classification_dir):
                        os.makedirs(classification_dir)

                    print "Running %s" % classification_dir
                    wf = ml_class(caps_dir, subjects_visits_tsv, diagnoses_tsv, group_id, image_type, atlas,
                                  classification_dir, pvc=pvc, n_iterations=n_iterations, n_threads=n_threads)
                    wf.run()


### Classification results using T1-weighted MRI from OASIS dataset

(from `1-main_classifications/OASIS_run_classifications.py`)

In [ ]:

import os
from os import path
import pandas as pd
import clinica.pipeline.machine_learning.ml_workflows as ml_wf

n_iterations = 250
n_threads = 8

caps_dir = '/OASIS/CAPS'
output_dir = '/OASIS/CLASSIFICATION/OUTPUTS'

group_id = 'OASISbl'
]image_types = ['T1'
tasks_dir = '/teams/ARAMIS/PROJECTS/simona.bottani/ADML_paper/OASIS/lists_by_task'
tasks = [('CN', 'AD')]

smoothing = [0, 4, 8, 12]
atlases = ['AAL2', 'LPBA40', 'Neuromorphometrics', 'AICHA', 'Hammers']
pvcs = [None, 'rbv']
rb_classifiers = {'linear_svm': ml_wf.RB_RepHoldOut_DualSVM,
                  'logistic_regression': ml_wf.RB_RepHoldOut_LogisticRegression,
                  'random_forest': ml_wf.RB_RepHoldOut_RandomForest}

###################################
### Voxel based classifications ###
###################################

for image_type in image_types:
    for fwhm in smoothing:
        for task in tasks:
            subjects_visits_tsv = path.join(tasks_dir, '%s_vs_%s_subjects_sessions.tsv' % (task[0], task[1]))
            diagnoses_tsv = path.join(tasks_dir, '%s_vs_%s_diagnoses.tsv' % (task[0], task[1]))
            for pvc in pvcs:
                if image_type == 'T1':
                    if pvc is None:
                        classification_dir = path.join(output_dir, image_type, 'voxel_based',
                                                       'smooothing-%s' % fwhm, 'linear_svm',
                                                       '%s_vs_%s' % (task[0], task[1]))
                    else:
                        continue
                else:
                    classification_dir = path.join(output_dir, image_type, 'voxel_based', 'pvc-%s' % pvc,
                                                   'smooothing-%s' % fwhm, 'linear_svm',
                                                   '%s_vs_%s' % (task[0], task[1]))
                if not path.exists(classification_dir):
                    os.makedirs(classification_dir)

                print "Running %s" % classification_dir
                wf = ml_wf.VB_RepHoldOut_DualSVM(caps_dir, subjects_visits_tsv, diagnoses_tsv, group_id, image_type,
                                                 classification_dir, fwhm=fwhm, pvc=pvc, n_iterations=n_iterations,
                                                 n_threads=n_threads)
                wf.run()

####################################
### Region based classifications ###
####################################

for image_type in image_types:
    for atlas in atlases:
        for task in tasks:
            subjects_visits_tsv = path.join(tasks_dir, '%s_vs_%s_subjects_sessions.tsv' % (task[0], task[1]))
            diagnoses_tsv = path.join(tasks_dir, '%s_vs_%s_diagnoses.tsv' % (task[0], task[1]))
            for classifier in rb_classifiers:
                ml_class = rb_classifiers[classifier]
                for pvc in pvcs:
                    if image_type == 'T1':
                        if pvc is None:
                            classification_dir = path.join(output_dir, image_type, 'region_based',
                                                           'atlas-%s' % atlas, classifier,
                                                           '%s_vs_%s' % (task[0], task[1]))
                        else:
                            continue
                    else:
                        classification_dir = path.join(output_dir, image_type, 'region_based', 'pvc-%s' % pvc,
                                                       'atlas-%s' % atlas, classifier,
                                                       '%s_vs_%s' % (task[0], task[1]))
                    if not path.exists(classification_dir):
                        os.makedirs(classification_dir)

                    print "Running %s" % classification_dir
                    wf = ml_class(caps_dir, subjects_visits_tsv, diagnoses_tsv, group_id, image_type, atlas,
                                  classification_dir, pvc=pvc, n_iterations=n_iterations, n_threads=n_threads)
                    wf.run()


Other more specific classification tasks can be run directly from the `Code` directory:

#### Comparison of classification results using T1-weighted MRI with different magnetic field strenghts from ADNI dataset

```
python 2-magnetic_field_strength/get_accuracies.py
```

#### Effect of class imbalance on classification results using T1-weighted MRI from ADNI and AIBL datasets

```
python 3-class_imbalance/subjects_ADNI.py
python 3-class_imbalance/subjects_AIBL.py
python 3-class_imbalance/1-ADNI_balanced_voxel_based.py
python 3-class_imbalance/2-ADNI_balanced_region_based.py
python 3-class_imbalance/3-AIBL_CN_AD_balanced.py
```



#### Effect of dataset on classification results and generalization tests using T1-weighted MRI from ADNI, AIBL and OASIS datasets

Generating subjects lists with the specific number of subjects for all the upcoming tasks:
```
python 4-influence_of_dataset/subjects.py
```

Obtaining dataset specific classification results with the same number of subjects:
```
python 4-influence_of_dataset/1-subset_70_subj/ADNI_run_classification_70_49.py
python 4-influence_of_dataset/1-subset_70_subj/AIBL_run_classification_70.py
python 4-influence_of_dataset/1-subset_70_subj/OASIS_run_classification_70.py
```

Generalization test of ADNI trained classifier on AIBL and OASIS (using all available subjects):
```
python 4-influence_of_dataset/2-full_ADNI_on_full_datasets/ADNI_on_AIBL_run_classifications_region.py
python 4-influence_of_dataset/2-full_ADNI_on_full_datasets/ADNI_on_AIBL_run_classifications_voxel.py
python 4-influence_of_dataset/2-full_ADNI_on_full_datasets/ADNI_on_OASIS_run_classifications_region.py
python 4-influence_of_dataset/2-full_ADNI_on_full_datasets/ADNI_on_OASIS_run_classifications_voxel.py
```

Generalization test of ADNI trained classifier on AIBL and OASIS (using same number of subjects):
```
python 4-influence_of_dataset/3-49s_ADNI_on_70s_datasets/ADNI_on_AIBL_run_classifications_region.py
python 4-influence_of_dataset/3-49s_ADNI_on_70s_datasets/ADNI_on_AIBL_run_classifications_voxel.py
python 4-influence_of_dataset/3-49s_ADNI_on_70s_datasets/ADNI_on_OASIS_run_classifications_region.py
python 4-influence_of_dataset/3-49s_ADNI_on_70s_datasets/ADNI_on_OASIS_run_classifications_voxel.py
```
